In [1]:
import os
import sys
import cv2
import pytesseract
from PIL import Image, ImageDraw,ImageFont
from pytesseract import Output

In [2]:
nb_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

import config
import src.utilities.app_context as app_context
app_context.init()

#path to craft model, weight can be pulled form the production bracnh of repo
config.CRAFT_MODEL_PATH= nb_dir + '/utilities/craft_pytorch/model/craft_mlt_25k.pth'
config.CRAFT_REFINE_MODEL_PATH = nb_dir + '/utilities/craft_pytorch/model/craft_refiner_CTW1500.pth'


In [3]:
from src.services.main import TextDetection

/home/naresh/ds-env2/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Loading weights of refiner from checkpoint (./model/craft_refiner_CTW1500.pth)
[2021-07-20 18:34:51,302] {geos.py:73} MainThread DEBUG in geos: Found GEOS DLL: <CDLL '/home/naresh/ds-env2/lib/python3.6/site-packages/shapely/.libs/libgeos_c-a68605fd.so.1.13.1', handle a5cb720 at 0x7f75fe2e3ef0>, using it.
[2021-07-20 18:34:51,318] {geos.py:38} MainThread DEBUG in geos: Trying `CDLL(libc.so.6)`
[2021-07-20 18:34:51,320] {geos.py:55} MainThread DEBUG in geos: Library path: 'libc.so.6'
[2021-07-20 18:34:51,321] {geos.py:56} MainThread DEBUG in geos: DLL: <CDLL 'libc.so.6', handle 7f769663b000 at 0x7f75fe2e3eb8>


In [4]:
#base_dir = '/home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input'
base_dir = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/'
filename    = 'hi_const255_hi.pdf'

#filename    = 'hamlet_1.pdf'
file_format = 'pdf'
language    =  'hi'

In [5]:
app_context.application_context = {
  "input":{
  "inputs": [
    {
      "file": {
        "identifier": "string",
        "name": filename,
        "type": file_format
      },
      "config": {
        "OCR": {
          "option": "HIGH_ACCURACY",
          "language": "hi"
        }
      }
    }
  ]}
}


In [6]:
config.LANGUAGE_LINE_THRESOLDS['hi']['text_threshold']=0.5
config.LANGUAGE_LINE_THRESOLDS['hi']['link_threshold']=0.35
config.LANGUAGE_LINE_THRESOLDS['hi']['low_text']=0.5

In [7]:
resp = TextDetection(app_context,base_dir)

[2021-07-20 18:34:51,398] {loghandler.py:48} MainThread DEBUG in loghandler: Block merger starting processing {'input': {'inputs': [{'file': {'identifier': 'string', 'name': 'hi_const255_hi.pdf', 'type': 'pdf'}, 'config': {'OCR': {'option': 'HIGH_ACCURACY', 'language': 'hi'}}}]}}
[2021-07-20 18:34:51,403] {loghandler.py:20} MainThread INFO in loghandler: created processing directories successfully /home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/hi_const255_hi_72b33f6e-6864-43e6-a0ef-f0c2cf5a48cf
[2021-07-20 18:34:51,407] {loghandler.py:20} MainThread INFO in loghandler: filepath /home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/hi_const255_hi.pdf, working_dir /home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/hi_const255_hi_72b33f6e-6864-43e6-a0ef-f0c2cf5a48cf
[2021-07-20 18:34:51,531] {loghandler.py:20} MainThread INFO in loghandler: Extracting 

/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/src/utilities/craft_pytorch/craft_utils.py:239: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  polys = np.array(polys)


Angle of tilt detected 0.07897561745897363 
[2021-07-20 18:36:25,135] {loghandler.py:20} MainThread INFO in loghandler: Time taken for text detection is 38.17522192001343/1time per page : 38.17522192001343
Angle of tilt after correction 0.07897561745897363 
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out
lines are sorting out


In [8]:
file_index = 0
page_index =0
filepath = resp['rsp']['outputs'][file_index]['page_info'][page_index]

save_dir = '/home/naresh/word_detector_benchmark/'
#resp['rsp']['outputs'][file_index]['pages'][page_index]['words'][0]

In [9]:

def draw_box(resp,filepath,save_dir, thresh,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][file_index]['pages'][page_index]['words']:
        #font = ImageFont.truetype("sans-serif.ttf", 30)
        draw.text((10, 10),thresh,(255,0,255))
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)
        
    
    save_filepath = os.path.join(save_dir, "bbox_"+str(thresh)+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [10]:
#local draw box
def draw_box(resp,filepath,save_dir,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][file_index]['pages'][page_index]['lines']:
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)

    save_filepath = os.path.join(save_dir, "bbox_"+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [11]:
image =draw_box(resp,filepath,save_dir,color="green", save=True)       

In [12]:
def draw_tess(filepath,save_dir):
    img = cv2.imread(filepath)
    name = filepath.split("/")[-1]
    print(name)
    h, w, c = img.shape
    boxes = pytesseract.image_to_boxes(img)
    #try:
    d = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(d['text'])
    for i in range(n_boxes):
        if int(d['conf'][i]) > 1:
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite(save_dir+name, img)
    print(save_dir+name)
#     except:
#         pass
    

In [13]:
#draw_tess(filepath,save_dir)